In [ ]:
import numpy as np                                                 
import pandas as pd                                                                                   
import seaborn as sns                                              
%matplotlib inline
sns.set()

from subprocess import check_output
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor


import os
print(os.listdir("../input"))

# read data from file
train = pd.read_csv("../input/train.csv") 
test = pd.read_csv("../input/test.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
print("train.shape ",train.shape)
print("test.shape ",test.shape)

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
train.describe()

In [ ]:
#check for null in training dataset
train.isnull().sum()

In [ ]:
#check for null in test dataset
test.isnull().sum()

In [ ]:
#save id of test data
test_id=test["id"]
test_id.shape

In [ ]:
#dropping few columns
train.drop(["id","homepage","imdb_id","belongs_to_collection","tagline","original_title","overview","genres","poster_path","production_countries", "production_companies","release_date","spoken_languages","title",'Keywords', 'cast', 'crew'], axis = 1,inplace = True)
test.drop(["id","homepage","imdb_id","belongs_to_collection","tagline","original_title","overview","genres","poster_path","production_countries", "production_companies","release_date","spoken_languages","title",'Keywords', 'cast', 'crew'], axis = 1,inplace = True)

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
train.original_language.unique()

In [ ]:
from sklearn.preprocessing import LabelEncoder
#label encode original language

le = LabelEncoder()
train['original_language'] = le.fit_transform(train['original_language'])
train.head(1)

In [ ]:
test['original_language'] = le.fit_transform(test['original_language'])
test.head(1)

In [ ]:
train.original_language.unique()

In [ ]:
train.status.unique()

In [ ]:
#label encode status
train['status'] = le.fit_transform(train['status'])
train.head(1)

In [ ]:
test.status.unique()

In [ ]:
#replace null values of status with "Released"
test.status.mode()

In [ ]:
test['status'].fillna("Released", inplace=True)

In [ ]:
test.isnull().sum()

In [ ]:
test['status'] = le.fit_transform(test['status'])
test.head(1)

In [ ]:
test.isnull().any()

In [ ]:
np.isnan(train).any()

In [ ]:
#replace null values with mode
train.runtime.mode()

In [ ]:
train['runtime'].fillna(90, inplace=True)

In [ ]:
test.runtime.mode()

In [ ]:
#replace null values with mode
test['runtime'].fillna(100, inplace=True)

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
sns.heatmap( train.corr(), annot=True )

In [ ]:
sns.distplot(train["budget"],axlabel="Distribution of budget")

In [ ]:
sns.distplot(train["revenue"],axlabel="Distribution of revenue")

In [ ]:
train.head(1)

In [ ]:
X_train=train[[ 'budget', 'original_language', 'popularity', 'runtime', 'status']] #feature columns
y_train=train.revenue #predictor variable

In [ ]:
#standardise the training data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train_std = scaler.transform(X_train)
X_train = pd.DataFrame(X_train_std)
X_train.columns = [ 'budget', 'original_language', 'popularity', 'runtime', 'status']
X_train.head()

In [ ]:
y_train.head()

In [ ]:
#standardise the test data
scaler = StandardScaler().fit(test)
test_std = scaler.transform(test)
test = pd.DataFrame(test_std)
test.columns = [ 'budget', 'original_language', 'popularity', 'runtime', 'status']
test.head()

In [ ]:
print(X_train.shape)
print(y_train.shape)

In [ ]:
#MOdel - Randaom Forest Regressor
model2 = RandomForestRegressor(random_state = 0)
model2.fit(X_train, y_train)
y_pred_train = model2.predict(X_train)
print(y_pred_train)

In [ ]:
print("Model Evaluation for Random Forest Regressor ")
RMSE_train = np.sqrt( metrics.mean_squared_error(y_train, y_pred_train))

#print('RMSE for training set is {}'.format(RMSE_train))

yhat = model2.predict(X_train)
SS_Residual = sum((y_train-yhat)**2)
SS_Total = sum((y_train-np.mean(y_train))**2)
r_squared = 1 - (float(SS_Residual))/SS_Total
adjusted_r_squared = 1 - (1-r_squared)*(len(y_train)-1)/(len(y_train)-X_train.shape[1]-1)
print("r_squared for train data ",r_squared, " and adjusted_r_squared for train data",adjusted_r_squared)


In [ ]:
test.head()

In [ ]:
#predict Revenue for test data
y_pred_test = model2.predict(test)
y_pred_test

In [ ]:
submission = pd.DataFrame({'id': test_id , 'revenue':y_pred_test})
submission = submission[['id', 'revenue']]
submission.head()

In [ ]:
submission.to_csv("submission.csv", index=False)